In [19]:
import numpy as np
import graphlearning as gl
from scipy.special    import gamma
from scipy.sparse import coo_matrix
from scipy.sparse.linalg import eigsh
from tqdm import tqdm


np.random.seed(0)
m=1
alpha = np.pi**(m/2) / gamma(m/2+1)
sigma = alpha / (m+2)
# density p = 1/((m+1)*area(S^m)); here area(S^2)=4π so this simplifies accordingly
p     = 1/((m+1) * np.pi**((m+1)/2) / gamma((m+1)/2+1))

total_n = 1000
k = int(total_n**(4/(m+4)))
num_vals = 5  # Compute a few eigenpairs

# Data generation on the unit circle (1-sphere)
estimations_1000 = []
for _ in tqdm(range(100)):
    X = gl.utils.rand_ball(total_n, m+1)
    X /= np.linalg.norm(X, axis=1)[:,None]

    # 5b) build k-NN graph Laplacian
    J, D    = gl.weightmatrix.knnsearch(X, k)
    W_knn   = gl.weightmatrix.knn(None, k, knn_data=(J,D), kernel='uniform')
    L_knn   = (2*p**(2/m)/sigma) * gl.graph(W_knn).laplacian() * ((total_n*alpha/k)**(1+2/m)) / total_n
    vals_knn, vecs_knn = eigsh(L_knn, k=num_vals, which='SM')

    # 5c) build ε-graph Laplacian
    eps      = np.min(np.max(D, axis=1))
    mask     = (D.flatten() <= eps)
    I_flat   = np.repeat(np.arange(total_n), k)[mask]
    J_flat   = J.flatten()[mask]
    W_eps    = coo_matrix((np.ones_like(J_flat), (I_flat, J_flat)), shape=(total_n,total_n)).tocsr()
    L_eps    = (2/p/sigma) * gl.graph(W_eps).laplacian() / (total_n * eps**(m+2))
    vals_eps, vecs_eps = eigsh(L_eps, k=num_vals, which='SM')

    # Choose eigenpairs
    eigenvalues, eigenvectors = vals_eps, vecs_eps

    j, k_idx = 1, 1  # Indices for eigenpairs (u_j, lambda_j), (u_k, lambda_k)
    hat_lambda_j, hat_u_j = eigenvalues[j], eigenvectors[:, j]
    hat_lambda_k, hat_u_k = eigenvalues[k_idx], eigenvectors[:, k_idx]

    # Define kernel eta (indicator kernel)
    def eta(t):
        return (t <= 1).astype(float)

    # Gradient estimator (as given)
    def G_hat(u, X, eps, m):
        total_n = X.shape[0]
        G = np.zeros(total_n)
        for i in range(total_n):
            distances = np.linalg.norm(X - X[i], axis=1)
            kernel_vals = eta(distances / eps)
            diff_squared = (u - u[i])**2
            G[i] = np.sum(diff_squared * kernel_vals) / (2 * total_n * eps**(2 + m)) 
        return G

    # Compute G_hat for eigenfunctions
    G_j_hat = G_hat(hat_u_j, X, eps, m)
    G_k_hat = G_hat(hat_u_k, X, eps, m)

    # Evaluate the proposed estimator
    estimator = (
        (hat_lambda_j * hat_u_j**2 + hat_lambda_j - 2 * G_j_hat)
        * (hat_lambda_k * hat_u_k**2 + hat_lambda_k - 2 * G_k_hat)
    ).mean()

    estimations_1000.append(estimator)

100%|██████████| 100/100 [00:20<00:00,  4.97it/s]


In [20]:
import numpy as np
import graphlearning as gl
from scipy.special    import gamma
from scipy.sparse import coo_matrix
from scipy.sparse.linalg import eigsh
from tqdm import tqdm


np.random.seed(0)
m=1
alpha = np.pi**(m/2) / gamma(m/2+1)
sigma = alpha / (m+2)
# density p = 1/((m+1)*area(S^m)); here area(S^2)=4π so this simplifies accordingly
p     = 1/((m+1) * np.pi**((m+1)/2) / gamma((m+1)/2+1))

total_n = 2000
k = int(total_n**(4/(m+4)))
num_vals = 5  # Compute a few eigenpairs

# Data generation on the unit circle (1-sphere)
estimations_2000 = []
for _ in tqdm(range(100)):
    X = gl.utils.rand_ball(total_n, m+1)
    X /= np.linalg.norm(X, axis=1)[:,None]

    # 5b) build k-NN graph Laplacian
    J, D    = gl.weightmatrix.knnsearch(X, k)
    W_knn   = gl.weightmatrix.knn(None, k, knn_data=(J,D), kernel='uniform')
    L_knn   = (2*p**(2/m)/sigma) * gl.graph(W_knn).laplacian() * ((total_n*alpha/k)**(1+2/m)) / total_n
    vals_knn, vecs_knn = eigsh(L_knn, k=num_vals, which='SM')

    # 5c) build ε-graph Laplacian
    eps      = np.min(np.max(D, axis=1))
    mask     = (D.flatten() <= eps)
    I_flat   = np.repeat(np.arange(total_n), k)[mask]
    J_flat   = J.flatten()[mask]
    W_eps    = coo_matrix((np.ones_like(J_flat), (I_flat, J_flat)), shape=(total_n,total_n)).tocsr()
    L_eps    = (2/p/sigma) * gl.graph(W_eps).laplacian() / (total_n * eps**(m+2))
    vals_eps, vecs_eps = eigsh(L_eps, k=num_vals, which='SM')

    # Choose eigenpairs
    eigenvalues, eigenvectors = vals_eps, vecs_eps

    j, k_idx = 1, 1  # Indices for eigenpairs (u_j, lambda_j), (u_k, lambda_k)
    hat_lambda_j, hat_u_j = eigenvalues[j], eigenvectors[:, j]
    hat_lambda_k, hat_u_k = eigenvalues[k_idx], eigenvectors[:, k_idx]

    # Define kernel eta (indicator kernel)
    def eta(t):
        return (t <= 1).astype(float)

    # Gradient estimator (as given)
    def G_hat(u, X, eps, m):
        total_n = X.shape[0]
        G = np.zeros(total_n)
        for i in range(total_n):
            distances = np.linalg.norm(X - X[i], axis=1)
            kernel_vals = eta(distances / eps)
            diff_squared = (u - u[i])**2
            G[i] = np.sum(diff_squared * kernel_vals) / (2 * total_n * eps**(2 + m)) 
        return G

    # Compute G_hat for eigenfunctions
    G_j_hat = G_hat(hat_u_j, X, eps, m)
    G_k_hat = G_hat(hat_u_k, X, eps, m)

    # Evaluate the proposed estimator
    estimator = (
        (hat_lambda_j * hat_u_j**2 + hat_lambda_j - 2 * G_j_hat)
        * (hat_lambda_k * hat_u_k**2 + hat_lambda_k - 2 * G_k_hat)
    ).mean()

    estimations_2000.append(estimator)

100%|██████████| 100/100 [01:16<00:00,  1.30it/s]


In [21]:
import numpy as np
import graphlearning as gl
from scipy.special    import gamma
from scipy.sparse import coo_matrix
from scipy.sparse.linalg import eigsh
from tqdm import tqdm


np.random.seed(0)
m=1
alpha = np.pi**(m/2) / gamma(m/2+1)
sigma = alpha / (m+2)
# density p = 1/((m+1)*area(S^m)); here area(S^2)=4π so this simplifies accordingly
p     = 1/((m+1) * np.pi**((m+1)/2) / gamma((m+1)/2+1))

total_n = 3000
k = int(total_n**(4/(m+4)))
num_vals = 5  # Compute a few eigenpairs

# Data generation on the unit circle (1-sphere)
estimations_3000 = []
for _ in tqdm(range(100)):
    X = gl.utils.rand_ball(total_n, m+1)
    X /= np.linalg.norm(X, axis=1)[:,None]

    # 5b) build k-NN graph Laplacian
    J, D    = gl.weightmatrix.knnsearch(X, k)
    W_knn   = gl.weightmatrix.knn(None, k, knn_data=(J,D), kernel='uniform')
    L_knn   = (2*p**(2/m)/sigma) * gl.graph(W_knn).laplacian() * ((total_n*alpha/k)**(1+2/m)) / total_n
    vals_knn, vecs_knn = eigsh(L_knn, k=num_vals, which='SM')

    # 5c) build ε-graph Laplacian
    eps      = np.min(np.max(D, axis=1))
    mask     = (D.flatten() <= eps)
    I_flat   = np.repeat(np.arange(total_n), k)[mask]
    J_flat   = J.flatten()[mask]
    W_eps    = coo_matrix((np.ones_like(J_flat), (I_flat, J_flat)), shape=(total_n,total_n)).tocsr()
    L_eps    = (2/p/sigma) * gl.graph(W_eps).laplacian() / (total_n * eps**(m+2))
    vals_eps, vecs_eps = eigsh(L_eps, k=num_vals, which='SM')

    # Choose eigenpairs
    eigenvalues, eigenvectors = vals_eps, vecs_eps

    j, k_idx = 1, 1  # Indices for eigenpairs (u_j, lambda_j), (u_k, lambda_k)
    hat_lambda_j, hat_u_j = eigenvalues[j], eigenvectors[:, j]
    hat_lambda_k, hat_u_k = eigenvalues[k_idx], eigenvectors[:, k_idx]

    # Define kernel eta (indicator kernel)
    def eta(t):
        return (t <= 1).astype(float)

    # Gradient estimator (as given)
    def G_hat(u, X, eps, m):
        total_n = X.shape[0]
        G = np.zeros(total_n)
        for i in range(total_n):
            distances = np.linalg.norm(X - X[i], axis=1)
            kernel_vals = eta(distances / eps)
            diff_squared = (u - u[i])**2
            G[i] = np.sum(diff_squared * kernel_vals) / (2 * total_n * eps**(2 + m)) 
        return G

    # Compute G_hat for eigenfunctions
    G_j_hat = G_hat(hat_u_j, X, eps, m)
    G_k_hat = G_hat(hat_u_k, X, eps, m)

    # Evaluate the proposed estimator
    estimator = (
        (hat_lambda_j * hat_u_j**2 + hat_lambda_j - 2 * G_j_hat)
        * (hat_lambda_k * hat_u_k**2 + hat_lambda_k - 2 * G_k_hat)
    ).mean()

    estimations_3000.append(estimator)

100%|██████████| 100/100 [03:36<00:00,  2.17s/it]


In [22]:
import numpy as np
import graphlearning as gl
from scipy.special    import gamma
from scipy.sparse import coo_matrix
from scipy.sparse.linalg import eigsh
from tqdm import tqdm


np.random.seed(0)
m=1
alpha = np.pi**(m/2) / gamma(m/2+1)
sigma = alpha / (m+2)
# density p = 1/((m+1)*area(S^m)); here area(S^2)=4π so this simplifies accordingly
p     = 1/((m+1) * np.pi**((m+1)/2) / gamma((m+1)/2+1))

total_n = 4000
k = int(total_n**(4/(m+4)))
num_vals = 5  # Compute a few eigenpairs

# Data generation on the unit circle (1-sphere)
estimations_4000 = []
for _ in tqdm(range(100)):
    X = gl.utils.rand_ball(total_n, m+1)
    X /= np.linalg.norm(X, axis=1)[:,None]

    # 5b) build k-NN graph Laplacian
    J, D    = gl.weightmatrix.knnsearch(X, k)
    W_knn   = gl.weightmatrix.knn(None, k, knn_data=(J,D), kernel='uniform')
    L_knn   = (2*p**(2/m)/sigma) * gl.graph(W_knn).laplacian() * ((total_n*alpha/k)**(1+2/m)) / total_n
    vals_knn, vecs_knn = eigsh(L_knn, k=num_vals, which='SM')

    # 5c) build ε-graph Laplacian
    eps      = np.min(np.max(D, axis=1))
    mask     = (D.flatten() <= eps)
    I_flat   = np.repeat(np.arange(total_n), k)[mask]
    J_flat   = J.flatten()[mask]
    W_eps    = coo_matrix((np.ones_like(J_flat), (I_flat, J_flat)), shape=(total_n,total_n)).tocsr()
    L_eps    = (2/p/sigma) * gl.graph(W_eps).laplacian() / (total_n * eps**(m+2))
    vals_eps, vecs_eps = eigsh(L_eps, k=num_vals, which='SM')

    # Choose eigenpairs
    eigenvalues, eigenvectors = vals_eps, vecs_eps

    j, k_idx = 1, 1  # Indices for eigenpairs (u_j, lambda_j), (u_k, lambda_k)
    hat_lambda_j, hat_u_j = eigenvalues[j], eigenvectors[:, j]
    hat_lambda_k, hat_u_k = eigenvalues[k_idx], eigenvectors[:, k_idx]

    # Define kernel eta (indicator kernel)
    def eta(t):
        return (t <= 1).astype(float)

    # Gradient estimator (as given)
    def G_hat(u, X, eps, m):
        total_n = X.shape[0]
        G = np.zeros(total_n)
        for i in range(total_n):
            distances = np.linalg.norm(X - X[i], axis=1)
            kernel_vals = eta(distances / eps)
            diff_squared = (u - u[i])**2
            G[i] = np.sum(diff_squared * kernel_vals) / (2 * total_n * eps**(2 + m)) 
        return G

    # Compute G_hat for eigenfunctions
    G_j_hat = G_hat(hat_u_j, X, eps, m)
    G_k_hat = G_hat(hat_u_k, X, eps, m)

    # Evaluate the proposed estimator
    estimator = (
        (hat_lambda_j * hat_u_j**2 + hat_lambda_j - 2 * G_j_hat)
        * (hat_lambda_k * hat_u_k**2 + hat_lambda_k - 2 * G_k_hat)
    ).mean()

    estimations_4000.append(estimator)

100%|██████████| 100/100 [04:58<00:00,  2.99s/it]


In [23]:
import numpy as np
import graphlearning as gl
from scipy.special    import gamma
from scipy.sparse import coo_matrix
from scipy.sparse.linalg import eigsh
from tqdm import tqdm


np.random.seed(0)
m=1
alpha = np.pi**(m/2) / gamma(m/2+1)
sigma = alpha / (m+2)
# density p = 1/((m+1)*area(S^m)); here area(S^2)=4π so this simplifies accordingly
p     = 1/((m+1) * np.pi**((m+1)/2) / gamma((m+1)/2+1))

total_n = 5000
k = int(total_n**(4/(m+4)))
num_vals = 5  # Compute a few eigenpairs

# Data generation on the unit circle (1-sphere)
estimations_5000 = []
for _ in tqdm(range(100)):
    X = gl.utils.rand_ball(total_n, m+1)
    X /= np.linalg.norm(X, axis=1)[:,None]

    # 5b) build k-NN graph Laplacian
    J, D    = gl.weightmatrix.knnsearch(X, k)
    W_knn   = gl.weightmatrix.knn(None, k, knn_data=(J,D), kernel='uniform')
    L_knn   = (2*p**(2/m)/sigma) * gl.graph(W_knn).laplacian() * ((total_n*alpha/k)**(1+2/m)) / total_n
    vals_knn, vecs_knn = eigsh(L_knn, k=num_vals, which='SM')

    # 5c) build ε-graph Laplacian
    eps      = np.min(np.max(D, axis=1))
    mask     = (D.flatten() <= eps)
    I_flat   = np.repeat(np.arange(total_n), k)[mask]
    J_flat   = J.flatten()[mask]
    W_eps    = coo_matrix((np.ones_like(J_flat), (I_flat, J_flat)), shape=(total_n,total_n)).tocsr()
    L_eps    = (2/p/sigma) * gl.graph(W_eps).laplacian() / (total_n * eps**(m+2))
    vals_eps, vecs_eps = eigsh(L_eps, k=num_vals, which='SM')

    # Choose eigenpairs
    eigenvalues, eigenvectors = vals_eps, vecs_eps

    j, k_idx = 1, 1  # Indices for eigenpairs (u_j, lambda_j), (u_k, lambda_k)
    hat_lambda_j, hat_u_j = eigenvalues[j], eigenvectors[:, j]
    hat_lambda_k, hat_u_k = eigenvalues[k_idx], eigenvectors[:, k_idx]

    # Define kernel eta (indicator kernel)
    def eta(t):
        return (t <= 1).astype(float)

    # Gradient estimator (as given)
    def G_hat(u, X, eps, m):
        total_n = X.shape[0]
        G = np.zeros(total_n)
        for i in range(total_n):
            distances = np.linalg.norm(X - X[i], axis=1)
            kernel_vals = eta(distances / eps)
            diff_squared = (u - u[i])**2
            G[i] = np.sum(diff_squared * kernel_vals) / (2 * total_n * eps**(2 + m)) 
        return G

    # Compute G_hat for eigenfunctions
    G_j_hat = G_hat(hat_u_j, X, eps, m)
    G_k_hat = G_hat(hat_u_k, X, eps, m)

    # Evaluate the proposed estimator
    estimator = (
        (hat_lambda_j * hat_u_j**2 + hat_lambda_j - 2 * G_j_hat)
        * (hat_lambda_k * hat_u_k**2 + hat_lambda_k - 2 * G_k_hat)
    ).mean()

    estimations_5000.append(estimator)

100%|██████████| 100/100 [07:19<00:00,  4.40s/it]


In [24]:
import numpy as np
import graphlearning as gl
from scipy.special    import gamma
from scipy.sparse import coo_matrix
from scipy.sparse.linalg import eigsh
from tqdm import tqdm


np.random.seed(0)
m=1
alpha = np.pi**(m/2) / gamma(m/2+1)
sigma = alpha / (m+2)
# density p = 1/((m+1)*area(S^m)); here area(S^2)=4π so this simplifies accordingly
p     = 1/((m+1) * np.pi**((m+1)/2) / gamma((m+1)/2+1))

total_n = 6000
k = int(total_n**(4/(m+4)))
num_vals = 5  # Compute a few eigenpairs

# Data generation on the unit circle (1-sphere)
estimations_6000 = []
for _ in tqdm(range(100)):
    X = gl.utils.rand_ball(total_n, m+1)
    X /= np.linalg.norm(X, axis=1)[:,None]

    # 5b) build k-NN graph Laplacian
    J, D    = gl.weightmatrix.knnsearch(X, k)
    W_knn   = gl.weightmatrix.knn(None, k, knn_data=(J,D), kernel='uniform')
    L_knn   = (2*p**(2/m)/sigma) * gl.graph(W_knn).laplacian() * ((total_n*alpha/k)**(1+2/m)) / total_n
    vals_knn, vecs_knn = eigsh(L_knn, k=num_vals, which='SM')

    # 5c) build ε-graph Laplacian
    eps      = np.min(np.max(D, axis=1))
    mask     = (D.flatten() <= eps)
    I_flat   = np.repeat(np.arange(total_n), k)[mask]
    J_flat   = J.flatten()[mask]
    W_eps    = coo_matrix((np.ones_like(J_flat), (I_flat, J_flat)), shape=(total_n,total_n)).tocsr()
    L_eps    = (2/p/sigma) * gl.graph(W_eps).laplacian() / (total_n * eps**(m+2))
    vals_eps, vecs_eps = eigsh(L_eps, k=num_vals, which='SM')

    # Choose eigenpairs
    eigenvalues, eigenvectors = vals_eps, vecs_eps

    j, k_idx = 1, 1  # Indices for eigenpairs (u_j, lambda_j), (u_k, lambda_k)
    hat_lambda_j, hat_u_j = eigenvalues[j], eigenvectors[:, j]
    hat_lambda_k, hat_u_k = eigenvalues[k_idx], eigenvectors[:, k_idx]

    # Define kernel eta (indicator kernel)
    def eta(t):
        return (t <= 1).astype(float)

    # Gradient estimator (as given)
    def G_hat(u, X, eps, m):
        total_n = X.shape[0]
        G = np.zeros(total_n)
        for i in range(total_n):
            distances = np.linalg.norm(X - X[i], axis=1)
            kernel_vals = eta(distances / eps)
            diff_squared = (u - u[i])**2
            G[i] = np.sum(diff_squared * kernel_vals) / (2 * total_n * eps**(2 + m)) 
        return G

    # Compute G_hat for eigenfunctions
    G_j_hat = G_hat(hat_u_j, X, eps, m)
    G_k_hat = G_hat(hat_u_k, X, eps, m)

    # Evaluate the proposed estimator
    estimator = (
        (hat_lambda_j * hat_u_j**2 + hat_lambda_j - 2 * G_j_hat)
        * (hat_lambda_k * hat_u_k**2 + hat_lambda_k - 2 * G_k_hat)
    ).mean()

    estimations_6000.append(estimator)

100%|██████████| 100/100 [10:29<00:00,  6.30s/it]


In [25]:
import numpy as np
import graphlearning as gl
from scipy.special    import gamma
from scipy.sparse import coo_matrix
from scipy.sparse.linalg import eigsh
from tqdm import tqdm


np.random.seed(0)
m=1
alpha = np.pi**(m/2) / gamma(m/2+1)
sigma = alpha / (m+2)
# density p = 1/((m+1)*area(S^m)); here area(S^2)=4π so this simplifies accordingly
p     = 1/((m+1) * np.pi**((m+1)/2) / gamma((m+1)/2+1))

total_n = 7000
k = int(total_n**(4/(m+4)))
num_vals = 5  # Compute a few eigenpairs

# Data generation on the unit circle (1-sphere)
estimations_7000 = []
for _ in tqdm(range(100)):
    X = gl.utils.rand_ball(total_n, m+1)
    X /= np.linalg.norm(X, axis=1)[:,None]

    # 5b) build k-NN graph Laplacian
    J, D    = gl.weightmatrix.knnsearch(X, k)
    W_knn   = gl.weightmatrix.knn(None, k, knn_data=(J,D), kernel='uniform')
    L_knn   = (2*p**(2/m)/sigma) * gl.graph(W_knn).laplacian() * ((total_n*alpha/k)**(1+2/m)) / total_n
    vals_knn, vecs_knn = eigsh(L_knn, k=num_vals, which='SM')

    # 5c) build ε-graph Laplacian
    eps      = np.min(np.max(D, axis=1))
    mask     = (D.flatten() <= eps)
    I_flat   = np.repeat(np.arange(total_n), k)[mask]
    J_flat   = J.flatten()[mask]
    W_eps    = coo_matrix((np.ones_like(J_flat), (I_flat, J_flat)), shape=(total_n,total_n)).tocsr()
    L_eps    = (2/p/sigma) * gl.graph(W_eps).laplacian() / (total_n * eps**(m+2))
    vals_eps, vecs_eps = eigsh(L_eps, k=num_vals, which='SM')

    # Choose eigenpairs
    eigenvalues, eigenvectors = vals_eps, vecs_eps

    j, k_idx = 1, 1  # Indices for eigenpairs (u_j, lambda_j), (u_k, lambda_k)
    hat_lambda_j, hat_u_j = eigenvalues[j], eigenvectors[:, j]
    hat_lambda_k, hat_u_k = eigenvalues[k_idx], eigenvectors[:, k_idx]

    # Define kernel eta (indicator kernel)
    def eta(t):
        return (t <= 1).astype(float)

    # Gradient estimator (as given)
    def G_hat(u, X, eps, m):
        total_n = X.shape[0]
        G = np.zeros(total_n)
        for i in range(total_n):
            distances = np.linalg.norm(X - X[i], axis=1)
            kernel_vals = eta(distances / eps)
            diff_squared = (u - u[i])**2
            G[i] = np.sum(diff_squared * kernel_vals) / (2 * total_n * eps**(2 + m)) 
        return G

    # Compute G_hat for eigenfunctions
    G_j_hat = G_hat(hat_u_j, X, eps, m)
    G_k_hat = G_hat(hat_u_k, X, eps, m)

    # Evaluate the proposed estimator
    estimator = (
        (hat_lambda_j * hat_u_j**2 + hat_lambda_j - 2 * G_j_hat)
        * (hat_lambda_k * hat_u_k**2 + hat_lambda_k - 2 * G_k_hat)
    ).mean()

    estimations_7000.append(estimator)

100%|██████████| 100/100 [14:04<00:00,  8.45s/it]


In [26]:
print(np.mean(estimations_1000))
print(np.mean(estimations_2000))
print(np.mean(estimations_3000))
print(np.mean(estimations_4000))
print(np.mean(estimations_5000))
print(np.mean(estimations_6000))
print(np.mean(estimations_7000))

0.9025645703248394
0.938026152497588
0.9536024535013987
0.9613205136123448
0.9694488731618367
0.97054533456495
0.9750559604003496


In [27]:
# Save the results
for i in range(1, 8):
    np.save(f'data_extension/estimations_{i*1000}.npy', eval(f'estimations_{i*1000}'))